In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
#This help auto print out the items without explixitly using 'print'
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/sample_submission.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=8/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
/kaggle/input/jane-street-real-time-market

## Initialize Pyspark framework and load data into Pyspark's dataframe

In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=ce02681ce67d746e94ec39300225ef98a54925ce02f0a584d29c8dd6f068531e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [5]:
# Import modules related to Pyspark
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains, isnan, udf, hour, array_min\
,array_max, countDistinct
from pyspark.sql.types import *

MAX_MEMORY = '57G'
#Initialize a spark session
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval',10000) \
        .set('spark.core.connection.ack.wait.timeout','3600') \
        .set('spark.executor.memory', MAX_MEMORY) \
        .set('spark.driver.memory', MAX_MEMORY)
def init_spark():
    spark = SparkSession.builder.appName('Pyspark for Jane Street EDA') \
            .config(conf=conf).getOrCreate()
    return spark
spark = init_spark()
filename_data = '/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet'
df = spark.read.parquet(filename_data, mode='DROPMALFORMED')
print('Data frame type: '+ str(type(df)))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/24 16:06:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Data frame type: <class 'pyspark.sql.dataframe.DataFrame'>


## Overview of Dataset
Schema, columns & datatypes of the dataset:

In [6]:
print('Data overview')
df.printSchema()
print('Colums overview')
pd.DataFrame(df.dtypes, columns = ['Column Name', 'Data type'])

Data overview
root
 |-- date_id: short (nullable = true)
 |-- time_id: short (nullable = true)
 |-- symbol_id: byte (nullable = true)
 |-- weight: float (nullable = true)
 |-- feature_00: float (nullable = true)
 |-- feature_01: float (nullable = true)
 |-- feature_02: float (nullable = true)
 |-- feature_03: float (nullable = true)
 |-- feature_04: float (nullable = true)
 |-- feature_05: float (nullable = true)
 |-- feature_06: float (nullable = true)
 |-- feature_07: float (nullable = true)
 |-- feature_08: float (nullable = true)
 |-- feature_09: byte (nullable = true)
 |-- feature_10: byte (nullable = true)
 |-- feature_11: short (nullable = true)
 |-- feature_12: float (nullable = true)
 |-- feature_13: float (nullable = true)
 |-- feature_14: float (nullable = true)
 |-- feature_15: float (nullable = true)
 |-- feature_16: float (nullable = true)
 |-- feature_17: float (nullable = true)
 |-- feature_18: float (nullable = true)
 |-- feature_19: float (nullable = true)
 |-- featur

,Column Name,Data type
0,date_id,smallint
1,time_id,smallint
2,symbol_id,tinyint
3,weight,float
4,feature_00,float
...,...,...
87,responder_4,float
88,responder_5,float
89,responder_6,float
90,responder_7,float


In [7]:
print('Data frame describe (string and numeric columns only):')
df.describe().toPandas()

print(f'There are total {df.count()} row, Let print first 2 data rows:')
df.limit(2).toPandas()

Data frame describe (string and numeric columns only):


24/10/24 16:11:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,count,5022952,5022952,5022952,5022952,5022952,5022952,5022952,5022952,5022952,...,5016753,5022952,5022952,5022952,5022952,5022952,5022952,5022952,5022952,5022952
1,mean,765.7737521680478,483.5,18.60994411254577,1.6525857040913814,-0.5267795208365678,-0.01738276676055358,-0.5281716258781135,-0.527570225069728,-0.007760018324231689,...,0.019522725057383627,-3.9596506314876633E-5,0.006478509572333006,3.2917702686604084E-5,-0.002725061302997164,-6.040429640939136E-4,-0.0030838396222724516,5.9417137212667596E-5,0.004988163674181543,-8.50881733677674E-4
2,stddev,48.793556019131735,279.4374089947359,11.287758916224343,0.9167079831223431,0.9606741321236238,1.0993381845604713,0.9586207384741229,0.9581990922400642,1.0674102962998053,...,1.297256635597856,0.6729324987739114,0.6203964129846808,0.6844648876528936,0.733359619559837,0.814491042051771,0.6291193242453755,0.9438185546229734,0.9636082860369369,0.93179334058048
3,min,680,0,0,0.20608176,-5.794129,-4.953487,-5.72601,-5.6018896,-5.7998796,...,-6.159515,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
4,max,849,967,38,7.619364,2.372838,4.4323664,2.3292744,2.538294,6.16416,...,257.454,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


There are total 5022952 row, Let print first 2 data rows:


,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,680,0,0,2.298160,0.851814,1.197591,0.219422,0.411698,2.057359,-0.542597,...,-0.195100,-0.304665,0.164485,-0.205231,0.191064,-1.413209,0.375675,0.929775,-1.574939,1.101371
1,680,0,1,3.928745,0.534441,1.079740,0.038748,0.275343,2.135057,-0.541966,...,-0.470271,0.089769,0.011395,0.092348,0.473781,0.397024,0.777026,0.826995,0.569681,1.986971


## Detect missing value and abnormal zeros

In [9]:
missing_values = {}
for index, column in enumerate(df.columns):
    missing_count = df.select(column).where((col(column).isNull()) | (col(column) == 0)).count()
    missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,5189,143264,0,0,0,0,0,0,0,...,6199,0,0,0,0,0,0,0,0,0


## Exploratory Data Analysis